# AI Project - Phase 2 (Destination Suggestion)
<div style="text-align: center">
<h1 style = "color: red"> Sharif University Of Technology</h1>
<h2 style = "color: green"> DR. Mahdieh Soleymani | DR. MohammadHossein Rohban </h2>
<h3 style = "color: cyan"> Head of Project: AmirHossein Razlighi <h3>
<h3 style = "color: cyan"> Designed By: AmirHossein Razlighi, Javad Hezareh, Payam Taebi, Alireza Sakhaei, Ali Banayeean, Yalda Shabanzadeh, Hamidreza Yaghoubi, Alireza Heidari <h3>
<h4 style = "color: white"> Ask your questions via quera</h4>
<h5> Save your file with format: STUDENT NUMBER_Phase2.ipynb or .zip </h5>
</div>

## Overview

<div style="text-align: center">
<img src="./Images/Uber_research.jpg" width="100%" height="auto" />
</div>

Suppose you are a research engineer at Uber and you are asked to design a system that suggests destinations to the passengers. The system should be able to suggest destinations based on the passenger's history. For example, one passenger may save a variety of locations (like home, work, gym, etc.). This passenger may go to gym, often on weekends and when he/she requests a car from Home.

So, for example, if I am a student, going to university usually from Saturday to Wednesday on 8:00 from "home", the next time I request a car from "home" on 8:00, the system should suggest "university" as the destination. Now, it's not that simple always, so we should seek for smart wayys to solve this problem!

As you may understood by now, we should engineer some of features from the data we have and try to predict the next destination of the passenger.

Let's begin!

## Setup

In [1]:
import random
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

random.seed(2024)
np.random.seed(2024)

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
root_path= './drive/MyDrive/AI_project_phase_1/ph2/'

# First Approach: Using KNN

## Data Preparation for KNN

In this part, you should prepare the data for KNN. You should load the dataset file that we provided, named `Data/output.json` and then clean it, do all the required preprocessings and then split into train-test-val sets if necessary. Note that we provided a splitted test set for you, named `Data/output_test.json`. You should not use this file for training or validation. You should only use it for testing your model.

In [4]:
# TODO: Load the dataset for training
df = pd.read_json(root_path+'Data/output.json')
df.head()

,user_id,Day,origin lat,origin lon,dest lat,dest lon,start_time,end_time,price
0,0,0,35.625,51.375,36.000,51.085,2024-01-30 13:07:00,2024-01-30 14:43:00,43.99
1,0,0,36.000,51.085,35.625,51.375,2024-01-30 15:13:00,2024-01-30 16:49:00,36.66
2,0,0,35.680,51.445,35.745,51.465,2024-01-30 20:41:00,2024-01-30 20:54:00,15.08
3,0,0,35.745,51.465,35.680,51.445,2024-01-30 21:24:00,2024-01-30 21:37:00,10.00
4,0,1,35.625,51.375,35.680,51.445,2024-01-30 23:38:00,2024-01-30 23:58:00,19.48


In [5]:
# TODO: Load the dataset for testing
df_test = pd.read_json(root_path+'Data/output_test.json')
df_test.head()

,user_id,Day,origin lat,origin lon,dest lat,dest lon,start_time,end_time,price
0,24,6,35.650,51.225,35.640,51.270,2024-01-30 11:58:00,2024-01-30 12:12:00,10.00
1,46,2,35.625,51.250,35.690,51.295,2024-01-30 21:58:00,2024-01-30 22:18:00,11.84
2,27,5,35.650,51.225,35.865,51.045,2024-01-30 20:05:00,2024-01-30 20:58:00,23.36
3,7,4,35.875,51.375,35.890,51.315,2024-01-30 17:39:00,2024-01-30 18:17:00,13.74
4,27,3,35.625,51.375,35.650,51.385,2024-01-30 13:16:00,2024-01-30 13:19:00,15.08


In [6]:
# print out the number of unique users and also randomly select one user
# TODO
print(df['user_id'].nunique())
sample_user_id = df['user_id'].sample().iloc[0]
print(sample_user_id)

60
50


In [7]:
!pip install keplergl
from keplergl import KeplerGl
# Uncomment the following line if you are using Google Colab
from google.colab import output
output.enable_custom_widget_manager()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 18.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922315 sha256=4134ec25a1f3540205fea84b5869f7e9b831f016381ad953a62543a4ccc89223
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl


In [8]:
# choose the data related to the randomly selected user
# show destinations of this user on map
# You should Use Kepler.gl to visualize the data

# TODO
sample_user_id = 31
user_df = df[df['user_id']==sample_user_id]
user_df_dests = user_df[['dest lat', 'dest lon']]
user_df_dests

,dest lat,dest lon
9161,35.625,51.250
9162,35.950,51.280
9163,35.820,51.225
9164,35.950,51.280
9165,35.625,51.250
...,...,...
9349,35.625,51.250
9350,35.650,51.260
9351,35.950,51.280
9352,35.875,51.275


In [9]:
# Sample output:
map_1 = KeplerGl()

# Add data to the map

map_1.add_data(data=user_df_dests, name='my_data')

# Visualize the map (opens in a new browser window)
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'my_data': {'index': [9161, 9162, 9163, 9164, 9165, 9166, 9167, 9168, 9169, 9170, 9171, 9172, 9…

Here, you should do feature engineering stuff! Extract the features that you think are important. Split the features into training set and also extract the related outputs (used for our model further). These outputs may be strings (name of destination) or destination's latitude/longitude or etc. Use your creativity!

In [10]:
# TODO
# import pandas as pd

# unique_combinations_labels = {}
# for i, (value1, value2) in enumerate(df[['dest lat', 'dest lon']].itertuples(index=False)):
#     if (value1, value2) not in unique_combinations_labels:
#       label = f'Label_{i + 1}'
#       unique_combinations_labels[(value1, value2)] = label

# df['dest_label'] = df.apply(lambda row: unique_combinations_labels.get((row['dest lat'], row['dest lon'])), axis=1)

# df.head()
def overfit_prep(df):
  df_pre = df.copy()
  df_pre['min'] = df_pre['start_time'].dt.minute
  df_pre['start_time'] = df_pre['start_time'] + pd.to_timedelta(30, unit='T')
  df_pre['hour'] = df_pre['start_time'].dt.hour
  df_pre = df.drop(['price', 'start_time'], axis = 1)
  return df_pre

def prep(df):
  df_pre = df.copy()
  df_pre['start_time'] = df_pre['start_time'] + pd.to_timedelta(30, unit='T')
  df_pre['hour'] = df_pre['start_time'].dt.hour
  df_pre = df.drop(['price', 'end_time', 'start_time'], axis = 1)
  return df_pre

df_train_pre = prep(df)
df_test_pre = prep(df_test)

df_train_pre

,user_id,Day,origin lat,origin lon,dest lat,dest lon
0,0,0,35.625,51.375,36.000,51.085
1,0,0,36.000,51.085,35.625,51.375
2,0,0,35.680,51.445,35.745,51.465
3,0,0,35.745,51.465,35.680,51.445
4,0,1,35.625,51.375,35.680,51.445
...,...,...,...,...,...,...
17989,59,0,35.950,51.225,35.875,51.125
17990,59,0,35.875,51.125,35.690,51.295
17991,59,0,35.875,51.125,35.525,51.100
17992,59,0,35.525,51.100,35.815,51.320


In [11]:
# print out the shape of the train and test dataframes
target = ['dest lat', 'dest lon']
y_train = df_train_pre[target].values
X_train = df_train_pre.drop(target, axis = 1).values

y_test = df_test_pre[target].values
X_test = df_test_pre.drop(target, axis = 1).values
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# TODO

(17994, 4) (17994, 2)
(4499, 4) (4499, 2)


## KNN Model

In this section, we are going to implement our KNN model. For further information on how KNN works, please refer to [this](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) link. As you can see, it's a simple algorithm. We will start with this and see the results of our _destination suggestion_ system.

In [12]:
import pandas as pd
import numpy as np

class NearestNeighbor:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors

    def fit(self, train_X, train_y):
        self.train_X = train_X
        self.train_y = train_y

    def predict(self, x):
        predictions = []
        for i in range(len(x)):
            distances = np.linalg.norm(self.train_X - x[i], axis=1)
            nearest_indices = np.argsort(distances)[:self.n_neighbors]

            unique_arrays, counts = np.unique(self.train_y[nearest_indices], axis=0, return_counts=True)
            most_repeated_index = np.argmax(counts)

            predictions.append(unique_arrays[most_repeated_index])

        return np.array(predictions)


In [13]:
# Fit the model on training data
KNN_model = NearestNeighbor(10)
KNN_model.fit(X_train, y_train)
# TODO

In [14]:
# from sklearn.metrics import accuracy_score
# Calculate the accuracy on training data
preds = KNN_model.predict(X_train)
def cal_acc(preds, y_train):
    cnt = 0
    for i in range(len(preds)):
        if all(preds[i] == y_train[i]):
            cnt += 1
    return cnt / len(preds)

cal_acc(preds, y_train)
# TODO

0.5661331554962765

In [15]:
# Calculate the accuracy on test data
preds = KNN_model.predict(X_test)
cal_acc(preds, y_test)
# TODO

0.5145587908424094

**QUESTION**: What do you think about this approach? Is it a good idea to use KNN for this problem? Why (or why not)? If the patterns in our datatset (passengers' history) get more complicated, will our model be robust to it in comparison to other models?

Your Answer:

In [16]:
# I fought with overfitting by reducing feature space, but if we choose bad features, the model will overfit on the training data.
# note that KNN performance will decrease in high dimensionalities. So it is not a good idea.
# As i said, KNN will not perform well in complex feature space. (because you are not learning general patterns. you are learning each individual trip)

# Second Approach: Using XGBoost

In this part, we are going to use XGBoost to predict the next destination of the passenger. You can use `xgboost` library to implement this model. To learn more about XGBoost, please refer to [this](https://en.wikipedia.org/wiki/XGBoost) link. It should be familiar to you, as you saw decision trees in the class.

For this part, you can use the same data (that you did all the processes on) from the previous part. Or, if you need, you can reload the dataset and do new preprocessings on it.

You can see the overview of how XGBoost works in the image below:

<dev style="text-align: center">
<img src="./Images/XGBoost.png" />
</dev>

Although, this is for more information and you **do not** need to implement `XGBoost` from scratch. You can use the library that we mentioned above.

In [17]:
%pip install xgboost

In [18]:
import xgboost as xgb

# Defining the model

class Encoder:
    def __init__(self):
        self.mapping = {}
        self.inverse_mapping = {}

    def fit(self, y):
        unique_values = np.unique(y, axis = 0)

        self.mapping = {tuple(val): idx for idx, val in enumerate(unique_values)}
        self.inverse_mapping = {idx: val for val, idx in self.mapping.items()}

    def transform(self, y):
        return np.array([self.mapping[tuple(val)] for val in y])

    def inverse_transform(self, enc):
        return np.array([self.inverse_mapping[idx] for idx in enc])



In [19]:
# TODO: fit the Encoder on training data
encoder = Encoder()
encoder.fit(y_train)

y_train_enc = encoder.transform(y_train)
y_test_enc = encoder.transform(y_test)

In [20]:
# TODO: transform the training and test data
# and fit the model on training data
classifier = xgb.XGBClassifier()
classifier.fit(X_train, y_train_enc)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [21]:
# TODO: print out train-data accuracy
train_preds = classifier.predict(X_train)
def cal_acc(preds, y_train):
    cnt = 0
    for i in range(len(preds)):
        if preds[i] == y_train[i]:
            cnt += 1
    return cnt / len(preds)

cal_acc(train_preds, y_train_enc)

0.6726686673335557

In [22]:
# TODO: print out test-data accuracy
test_preds = classifier.predict(X_test)
cal_acc(test_preds, y_test_enc)

0.5730162258279617

**QUESTION**: What do you think about this approach? Is it a good idea to use XGBoost for this problem? Why (or why not)? If the patterns in our datatset (passengers' history) get more complicated, will our model be robust to it in comparison to other models?

Your Answer: AS I mentioned below, if the data gets too complex, xgboost starts to memorizing the training data and it will overfit on the training set.
xgboost in gerenal is a very good model but our data here is complex and it can't extract all the patterns and it has to memorize training data so it's performance is not well on test set.
the reason KNN and xgboost acc is low on training, is because I just selected 4 or 5 features and they can't memorize the training becaue features are too simple and they can't overfit quite well.


**Question**: Please explain the problem of overfitting in XGBoost. How can you solve it? Provide a brief explanation.

Your Answer: ...

In [23]:
# by a good feature selection. if data gets too complex, xgboost starts to memorizing training data
# max dapth limit for the trees
# good learning rate: in xgboost methods learning rate controls contribution of each tree to final result which is quite important
# in boosting methods, increasing the number of estimators may increase the chance of overfitting

# Third Approach: Classifier Using Neural Networks

In this approach, we are going to use a classifier using neural networks. You can think of this approach and how to model the problem as a classification problem in many ways! So, we are not going to restrict your creativity. Just a hint: You can consider each of the unique destinations (in whole dataset) as a class and then train a classifier to classify the destinations.

## Data for Neural Network

Let's prepare the data for our neural network. Again, you should extract required features from the dataset and then split the dataset into train-test-val sets if necessary. For your ease, we prepared another version of `output.json` that helps you to extract features for this part, easier. So, please load `Data/trip_data.json` and use it for training set and validation set. You should use `Data/trip_data_test.json` for testing your model.

In [24]:
# load the training dataset
# TODO
df = pd.read_json(root_path + 'Data/trip_data.json')
df.head()

,user_id,Day,origin,destination,start_time,end_time,price
0,0,0,"[work, [35.625, 51.375]]","[restaurant, [36.0, 51.085]]",2024-01-30 13:07:00,2024-01-30 14:43:00,43.99
1,0,0,"[restaurant, [36.0, 51.085]]","[work, [35.625, 51.375]]",2024-01-30 15:13:00,2024-01-30 16:49:00,36.66
2,0,0,"[home, [35.68, 51.445]]","[restaurant, [35.745, 51.465]]",2024-01-30 20:41:00,2024-01-30 20:54:00,15.08
3,0,0,"[restaurant, [35.745, 51.465]]","[home, [35.68, 51.445]]",2024-01-30 21:24:00,2024-01-30 21:37:00,10.00
4,0,1,"[work, [35.625, 51.375]]","[home, [35.68, 51.445]]",2024-01-30 23:38:00,2024-01-30 23:58:00,19.48


In [25]:
# load the test dataset
# TODO
df_test = pd.read_json(root_path + 'Data/trip_data_test.json')
df_test.head()

,user_id,Day,origin,destination,start_time,end_time,price
0,24,6,"[pool, [35.65, 51.225]]","[home, [35.64, 51.27]]",2024-01-30 11:58:00,2024-01-30 12:12:00,10.00
1,46,2,"[university, [35.625, 51.25]]","[restaurant, [35.69, 51.295]]",2024-01-30 21:58:00,2024-01-30 22:18:00,11.84
2,27,5,"[pool, [35.65, 51.225]]","[restaurant, [35.865, 51.045]]",2024-01-30 20:05:00,2024-01-30 20:58:00,23.36
3,7,4,"[work, [35.875, 51.375]]","[home, [35.89, 51.315]]",2024-01-30 17:39:00,2024-01-30 18:17:00,13.74
4,27,3,"[work, [35.625, 51.375]]","[restaurant, [35.65, 51.385]]",2024-01-30 13:16:00,2024-01-30 13:19:00,15.08


In [26]:
def convert(df):
    df['origin lat'] = df['origin'].apply(lambda x: x[1][0])
    df['origin long'] = df['origin'].apply(lambda x: x[1][1])
    df['origin'] = df['origin'].apply(lambda x: x[0])
    # df['dest lat'] = df['destination'].apply(lambda x: x[1][0])
    # df['dest long'] = df['destination'].apply(lambda x: x[1][1])
    df['target'] = df['destination'].apply(lambda x: x[0])
    df['shour'] = df['start_time'].dt.hour
    df['smin'] = df['start_time'].dt.minute
    df = df.drop(['start_time', 'end_time', 'price', 'destination'], axis = 1)

    return df


In [27]:
# Extract features and do preprocessing if needed
df = convert(df)
df.head()
# TODO

,user_id,Day,origin,origin lat,origin long,target,shour,smin
0,0,0,work,35.625,51.375,restaurant,13,7
1,0,0,restaurant,36.000,51.085,work,15,13
2,0,0,home,35.680,51.445,restaurant,20,41
3,0,0,restaurant,35.745,51.465,home,21,24
4,0,1,work,35.625,51.375,home,23,38


In [28]:
# Extract features and do preprocessing if needed
# TODO
df_test = convert(df_test)
df_test.head()

,user_id,Day,origin,origin lat,origin long,target,shour,smin
0,24,6,pool,35.650,51.225,home,11,58
1,46,2,university,35.625,51.250,restaurant,21,58
2,27,5,pool,35.650,51.225,restaurant,20,5
3,7,4,work,35.875,51.375,home,17,39
4,27,3,work,35.625,51.375,restaurant,13,16


In [29]:
# print out the unique values for each column
# TODO
for column in df.columns:
    unique_values = df[column].unique()
    print(f'column {column}: {unique_values}\n')

column user_id: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59]

column Day: [0 1 2 3 4 6 5]

column origin: ['work' 'restaurant' 'home' 'park' 'university' 'gym' 'pool']

column origin lat: [35.625 36.    35.68  35.745 35.705 35.815 35.75  35.65  35.675 35.69
 35.785 35.995 35.875 35.5   35.805 35.95  35.575 35.505 35.56  35.51
 35.565 35.54  35.52  35.91  35.735 35.76  35.55  35.82  35.765 35.945
 35.7   35.97  35.865 35.74  35.955 35.725 35.8   35.525 35.98  35.62
 35.81  35.635 35.975 35.89  35.615 35.59  35.535 35.77  35.555 35.84
 35.845 35.64  35.595 35.99  35.775 35.53  35.855 35.71  35.545 35.9
 35.93  35.905 35.825 35.83 ]

column origin long: [51.375 51.085 51.445 51.465 51.295 51.32  51.385 51.31  51.37  51.24
 51.    51.045 51.275 51.315 51.5   51.345 51.185 51.42  51.18  51.415
 51.155 51.06  51.285 51.19  51.4   51.225 51.075 51.17  51.1

In [30]:
import pandas as pd

# print out the data type of each column
# TODO
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17994 entries, 0 to 17993
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      17994 non-null  int64  
 1   Day          17994 non-null  int64  
 2   origin       17994 non-null  object 
 3   origin lat   17994 non-null  float64
 4   origin long  17994 non-null  float64
 5   target       17994 non-null  object 
 6   shour        17994 non-null  int64  
 7   smin         17994 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 1.1+ MB


Based on your features and how you extracted them, you may need to use some encodings for your data. For example, if you have different classes as names (`str` data type. E.g. "gym") you need to make it a numeric value in order to feed it into your neural network. You can use `sklearn`'s functions (such as `LabelEncoder`, `OneHotEncoder`, `StandardScaler`, etc.) to do these kind of stuff!

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Finalize the data (do all preprocessing needed)
# TODO



target_encoder = LabelEncoder()

df['target'] = target_encoder.fit_transform(df['target'])
df_test['target'] = target_encoder.transform(df_test['target'])

df_train_y = df['target'].copy()
df_train_x = df.drop('target', axis = 1).copy()

df_test_y = df_test['target'].copy()
df_test_x = df_test.drop('target', axis = 1).copy()


In [32]:
df_train_xd = pd.get_dummies(df_train_x, columns=['origin', 'Day'])
df_test_xd = pd.get_dummies(df_test_x, columns=['origin', 'Day'])

In [33]:
scaler = StandardScaler()
df_train_ready_x = pd.DataFrame(scaler.fit_transform(df_train_xd), columns=df_train_xd.columns)
df_test_ready_x = pd.DataFrame(scaler.transform(df_test_xd), columns=df_test_xd.columns)

In [34]:
df_train_ready_y = pd.get_dummies(df_train_y)
df_test_ready_y = pd.get_dummies(df_test_y)

In [35]:
X_train = df_train_ready_x.values
y_train = df_train_ready_y.values

X_test = df_test_ready_x.values
y_test = df_test_ready_y.values
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)

(4499, 19) (4499, 7)
(17994, 19) (17994, 7)


Create `train_dataset` and its loader, also create `test_dataset` and its loader. You should also create `val_dataset` and its loader if you want to use validation set. You may need to implement a custom `torch.Dataset` class for your ease. Your loaders should be able to load data in batches.

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## Neural Network Model

Here, you should implement your neural network model. You should use `pytorch`. **Note** that you should plot the loss function of your model during the training phase. (on both training and validation sets)

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.fc3(self.relu2(x))
        return x

input_dim = 19
output_dim = 7
model = MLP(input_dim=input_dim, output_dim=output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_params = None

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_params = model.state_dict()
    return best_model_params

    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
    plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()



In [38]:
best_params = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=100)

Epoch 1/100, Train Loss: 0.8584, Val Loss: 0.5673
Epoch 2/100, Train Loss: 0.4901, Val Loss: 0.4593
Epoch 3/100, Train Loss: 0.4168, Val Loss: 0.3964
Epoch 4/100, Train Loss: 0.3723, Val Loss: 0.3533
Epoch 5/100, Train Loss: 0.3426, Val Loss: 0.3267
Epoch 6/100, Train Loss: 0.3248, Val Loss: 0.3131
Epoch 7/100, Train Loss: 0.3116, Val Loss: 0.3281
Epoch 8/100, Train Loss: 0.3024, Val Loss: 0.3055
Epoch 9/100, Train Loss: 0.2961, Val Loss: 0.2958
Epoch 10/100, Train Loss: 0.2907, Val Loss: 0.2874
Epoch 11/100, Train Loss: 0.2856, Val Loss: 0.3011
Epoch 12/100, Train Loss: 0.2824, Val Loss: 0.2938
Epoch 13/100, Train Loss: 0.2785, Val Loss: 0.2884
Epoch 14/100, Train Loss: 0.2771, Val Loss: 0.2823
Epoch 15/100, Train Loss: 0.2746, Val Loss: 0.2776
Epoch 16/100, Train Loss: 0.2699, Val Loss: 0.2701
Epoch 17/100, Train Loss: 0.2666, Val Loss: 0.2737
Epoch 18/100, Train Loss: 0.2664, Val Loss: 0.2666
Epoch 19/100, Train Loss: 0.2643, Val Loss: 0.2789
Epoch 20/100, Train Loss: 0.2618, Val Lo

## Evaluate Neural Network Model

Report the model's accuracy on the test set.

In [39]:
# report model accuracy on test data
# TODO
def calculate_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            _, labels = torch.max(labels, 1)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

model.load_state_dict(best_params)
calculate_accuracy(model, test_loader)

0.8899755501222494

Just for the sake of curiosity, let's take a random sample from the test set and see the model's prediction. So, randomly choose a sample from the test set and print it out (to see its features and also the correct output). Then, feed the features into your model and see what it predicts. Is it correct?

In [40]:
import random
# TODO: choose a random sample from test data and print it out
rsi = random.randint(0, len(df_test_ready_x)-1)
rs = df_test_ready_x.iloc[rsi]
rs_label = df_test_ready_y.iloc[rsi]
rs

user_id              0.753298
origin lat          -0.035854
origin long          1.688047
shour                0.183275
smin                -1.533597
origin_gym          -0.181033
origin_home         -0.669397
origin_park         -0.179554
origin_pool         -0.180869
origin_restaurant    1.587219
origin_university   -0.245773
origin_work         -0.584752
Day_0               -0.452126
Day_1               -0.433813
Day_2               -0.429191
Day_3               -0.429555
Day_4                2.312378
Day_5               -0.335244
Day_6               -0.336985
Name: 3850, dtype: float64

In [41]:
# TODO: predict the destination of the above sample
# print out the prediction
data = rs.values
model.eval()
with torch.no_grad():
    data = torch.FloatTensor(data)
    outputs = model(data)
    _, pred = torch.max(outputs, 0)

pred




tensor(6)

In [42]:
# TODO: print out the actual destination of the above sample
np.argmax(rs_label)


6

Now, use the inverse transform of the encoding you used earlier to get the name of the destination from the predicted class. Print it out and see if it's correct.

In [43]:
# TODO: use inverse_transform to print out the actual name of destination of the above sample
res = pred.detach().numpy()
decoded_data = target_encoder.inverse_transform([res])
decoded_data

array(['work'], dtype=object)

**QUESTION**: What do you think about this approach? Is it a good idea to use Neural Networks for this problem? Why (or why not)? If the patterns in our datatset (passengers' history) get more complicated, will our model be robust to it in comparison to other models?

Your Answer: this model performed well and appeared to be a good estimator because it is very powerfull in estimating complex functions. it will be robust because we monitored it's learning with val set and it didn't overfitt on the training and therefore perfomed well on the test set. So it is robust because we can say that it didn't memorize the data and learned the patterns truely.

# How to Submit:

Please upload your notebook (`.ipynb`) compressed (as a `.zip` file) or uncompressed on Quera. **Note** that for each part, the accuracy of your model on the given test dataset is important. So, your accuracy should be **at least** same as ours (or better). Also, we will check your code after the submission. So, please make sure that there are no **data snooping** or **data leakage** in your code. You **can not** use the test data in any stages for your model, except for the final evaluation part! So, please be ware of that, or you may lose points.

Your project is graded via 2 main parts:
1. Checking out your implementation (to check if there are no "data snooping" and "data leakage")
   
2. Checking accuracies of your models on the test set.

3. Running again your code by you in the "in-person" session and hearing your explanations (on the parts that we asked questions about the models, overfitting and etc. with tag "QUESTION" in the notebook).